# Harvard CS 181
## Section 3 Demo

Run the code below to import the necessary libraries


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib inline
from IPython.display import HTML

## Gradient Descent

The code for this section has beeen sourced from [this tutorial](https://jed-ai.github.io/py1_gd_animation)

The algorithm is run for *max_epochs* of epochs or until the absolute value of improvement in our objective function falls below a certain threshold called *tolerance*, whichever comes earlier. 


In [0]:
def gradient_descent(f, df, lr, x0, max_epochs = 10, tolerance = .0001):
    """Perform gradient descent and return lists of points along trajectory
    f  : function to minimize
    df : gradient function of f
    lr : learning rate
    x0 : initial guess for x
    max_epochs : max number of epochs to run gradient descent for
    tolerance : if abs change in f falls below this value, stop grad descent
    """
    prev_x = x0
    xs = np.array([prev_x])
    ys = np.array([f(prev_x)])
    
    y_del = np.inf
    epochs = 0
    while epochs < max_epochs and y_del > tolerance:
        new_x = prev_x - lr * df(prev_x)
        xs = np.append(xs, new_x)
        ys = np.append(ys, f(new_x))
        y_del = abs(f(prev_x) - f(new_x))
        prev_x = new_x
        epochs += 1
        
    return xs, ys

In [0]:
def animate_gd(f, df, lr, x0, max_epochs = 10, tolerance = .0001):
    """animate gradient descent trajectory
    f  : function to minimize
    df : gradient function of f
    lr : learning rate
    x0 : initial guess for x
    max_epochs : max number of epochs to run gradient descent for
    tolerance : if change in f falls below this value, stop grad descent
    """
    
    # obtain trajectory points
    xs, ys = gradient_descent(f, df, lr, x0, max_epochs, tolerance)
    
    # x and y for plotting function
    x = np.arange(-1, 5, 0.01)
    y = f(x)
    
    # create plot
    fig, ax = plt.subplots()
    
    # plot funtion
    ax.plot(x, y, lw = 0.9, color = 'k')
    ax.set_xlabel(r'$x$')
    ax.set_ylabel(r'$y$')
    
    #prepare for animation
    line, = ax.plot([], [], 'r', label = 'Gradient Descent', lw = 1.5)
    point, = ax.plot([], [], 'bo')
    value_display = ax.text(0.02, 0.02, '', transform=ax.transAxes)

    def init():
        line.set_data([], [])
        point.set_data([], [])
        value_display.set_text('')
        return line, point, value_display

    def animate(i):
        # Animate line
        line.set_data(xs[:i], ys[:i])

        # Animate points
        point.set_data(xs[i], ys[i])

        # Animate value display
        value_display.set_text('Min = ' + str(ys[i]))

        return line, point, value_display

    ax.legend(loc = 1)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=len(xs), interval=120, 
                                   repeat_delay=60, blit=True)
    plt.close()

    return HTML(anim.to_html5_video())

### Examples

We will explore an example function 
$$f(x) = x^2 - 4x + 2$$

This is an upward facing parabola with a single minimum (convex function). 

Its derivative is 
$$\frac{d}{dx}f(x) = 2x - 4$$

In [0]:
def f(x):
    f_x = x**2 - 4*x + 2
    return f_x

def df(x):
    df_x = 2*x - 4
    return df_x

In [0]:
x0 = -1
max_epochs = 100
tolerance = 0.00001

To be conservative,  let's start with a very small learning rate of 0.01

In [0]:
lr = 0.01
animate_gd(f, df, lr, x0, max_epochs, tolerance)

We are able to find the minimum successfully but it takes a large number of epochs to reach it.

Lets increase the learning rate to 0.1

In [0]:
lr = .1
animate_gd(f, df, lr, x0, max_epochs, tolerance)

This trajectory looks nice but can we go even further?

Let's try a learning rate of 1



In [0]:
lr = 1i
animate_gd(f, df, lr, x0, max_epochs, tolerance)

This was bad. We are overshooting the minimum and oscillating between two points.

Lets look at another example with learning rate of 0.7

In [0]:
lr = .9
animate_gd(f, df, lr, x0, max_epochs, tolerance)

This trajectory eventually finds the minimum but it overshoots at each epoch. We got lucky here because the function we are minimizing is convex but not all functions are this nice as we will see later in the course. 

The choice of learning rate is important in optimization problems and requires tuning through heuristics and/or trial and error.



## One-hot vectors

### Motivation

We often come across functions of the form
$$\mathcal{L(y,z)} = \sum_{n=0}^{N-1}\sum_{k = 0}^{K-1}\mathbb{1}(y_n = k)z_k$$

where $y \in \{0, 1, \dots, K-1\}$ and $z$ is a $K$-dimensional vector.

We could code up this function using for loops and if else conditions as shown below but it gets messy fast and become computationally heavy as $N$ and $K$ increase.



In [0]:
K = 10
N = 20

z = np.random.rand(K)
z /= z.sum()

y = np.random.choice(K,N)

In [0]:
def L(y, z, N, K):
    l = 0
    for n in range(N):
        for k in range(N):
            if y[n] == k:
                l += z[k]
    return l

L(y, z, N, K)

1.9576661877138095

### One-hot Encodings
This is where one-hot encoding encoding comes in.

The one-hot encoding of $x \in \{0, 1, \dots, K-1\}$ is a $K$-dimensional vector where only one component (corresponding to the value of $x$) is 1 and all others are 0. It can be thought of as a feature vector of binary values whose elements are 1 if the $x$ belongs to the corresponding class and 0 otherwise.

So a datapoint $x = 2$ where $K=3$ would be represented using the one-hot vector 
$$\tilde{x} = \begin{pmatrix}0 \\ 0\\ 1\end{pmatrix}$$


In this notation, 

\begin{align*}
\mathcal{L(y,z)} &= \sum_{n=0}^{N-1}\sum_{k = 0}^{K-1}y_{n,k}z_k \\
&= \sum_{n=0}^{N-1}\tilde{y}_nz\end{align*}

where $\tilde{y}$ is a matrix whose rows are the transpose of the one-hot encodings of the corresponding componets of $y$.


This can be implemented using the code below which is more elegant and potentially faster since it can take advantage of optimized matrix operations.

In [0]:
def one_hottify(y, K):
    return np.eye(K)[y]

def L2(y, z, N, K):
    y_oh = one_hottify(y, K)
    return np.dot(y_oh, z).sum()
    
L2(y, z, N, K)

1.9576661877138095

One-hot vectors are also useful for other computations such as counting the number of occurances of each class.

In [0]:
num_occurances = one_hottify(y, K).sum(0)
num_occurances

array([0., 1., 0., 4., 1., 5., 1., 2., 4., 2.])

### Bonus
There is another efficient way to compute $\mathcal{L}$ which uses smart indexing and is equivalent to
$$\mathcal{L(y,z)} = \sum_{n=0}^{N-1}z_{y_n} $$

Although convenient for implementation, this notation isn't suited for mathematical purposes.

In [0]:
def L3(y, z):
    return z[y].sum()

L3(y,z)

1.9576661877138095